# Análisis de Ventas – Base de Datos Northwind

## Descripción general
La base de datos **Northwind** corresponde a una empresa ficticia dedicada a la comercialización y distribución de productos alimenticios a clientes internacionales.

Este proyecto tiene como objetivo analizar el comportamiento de las ventas, identificar los principales generadores de ingresos y evaluar el desempeño comercial desde diferentes dimensiones del negocio.

## Estructura de la base de datos

La base de datos está compuesta por las siguientes tablas principales:

- **Customers**: información de los clientes
- **Orders**: registro de las órdenes de venta
- **OrderDetails**: detalle de productos y cantidades por orden
- **Products**: catálogo de productos y precios
- **Categories**: clasificación de productos
- **Employees**: empleados responsables de las ventas
- **Shippers**: empresas encargadas del envío

Las relaciones entre tablas permiten analizar las ventas desde múltiples perspectivas: producto, cliente, país, categoría, empleado y tiempo.

## Enfoque del análisis

El análisis se desarrolla en dos etapas:

1. **Exploración inicial**  
   Comprensión del tamaño del negocio, volumen de ventas y periodo analizado.

2. **Análisis de negocio**  
   Evaluación de ingresos, productos más vendidos, desempeño por categorías, países, empleados y evolución temporal de las ventas.

Todas las métricas se calculan utilizando **SQL ejecutado desde Python**, permitiendo un análisis reproducible y documentado.

In [1]:
import sqlite3
import pandas as pd

# Conexión a la base de datos SQLite
db_path = "database/northwind.db"
conn = sqlite3.connect(db_path)

In [2]:
## Función para ejecutar consultas SQL (Code)

def run_query(query):
    return pd.read_sql_query(query, conn)

## 1. Exploración inicial de la base de datos

Antes de analizar el desempeño comercial, es necesario comprender la dimensión general del negocio y el volumen de información disponible.

In [3]:
query = """
SELECT 'Customers' AS Tabla, COUNT(*) AS Total FROM Customers
UNION ALL
SELECT 'Orders', COUNT(*) FROM Orders
UNION ALL
SELECT 'OrderDetails', COUNT(*) FROM OrderDetails
UNION ALL
SELECT 'Products', COUNT(*) FROM Products
UNION ALL
SELECT 'Categories', COUNT(*) FROM Categories
UNION ALL
SELECT 'Employees', COUNT(*) FROM Employees
UNION ALL
SELECT 'Shippers', COUNT(*) FROM Shippers;
"""

run_query(query)

,Tabla,Total
0,Customers,91
1,Orders,196
2,OrderDetails,518
3,Products,77
4,Categories,8
5,Employees,10
6,Shippers,3


**Análisis:**

La base de datos contiene información de 91 clientes y 196 órdenes de venta, con 518 registros de detalle, lo que indica un volumen suficiente para realizar un análisis exploratorio del negocio.

El catálogo está compuesto por 77 productos organizados en 8 categorías, gestionados por un equipo reducido de empleados.

## 2. KPIs generales del negocio

En esta sección se calculan los principales indicadores de desempeño (KPIs) que permiten evaluar el estado general del negocio desde una perspectiva de ventas.

In [4]:
query = """
SELECT
    ROUND(SUM(od.Quantity * p.Price), 2) AS VentasTotales,
    COUNT(DISTINCT o.OrderID) AS TotalOrdenes,
    COUNT(DISTINCT o.CustomerID) AS TotalClientes,
    ROUND(SUM(od.Quantity * p.Price) / COUNT(DISTINCT o.OrderID), 2) AS TicketPromedio,
    ROUND(SUM(od.Quantity * p.Price) / COUNT(DISTINCT o.CustomerID), 2) AS IngresoPromedioPorCliente
FROM Orders o
JOIN OrderDetails od ON o.OrderID = od.OrderID
JOIN Products p ON od.ProductID = p.ProductID;
"""
run_query(query)

,VentasTotales,TotalOrdenes,TotalClientes,TicketPromedio,IngresoPromedioPorCliente
0,386424.23,196,74,1971.55,5221.95


**Análisis:**

El negocio registra ventas totales por 386,424.23, distribuidas en 196 órdenes realizadas por 74 clientes activos.

El ticket promedio es de aproximadamente 1,971.55, lo que sugiere que las órdenes suelen incluir múltiples productos o artículos de precio medio-alto.

El ingreso promedio por cliente es de 5,221.95, indicando una relación comercial recurrente con los clientes activos.

## 3. Análisis de productos

En esta sección se analiza el desempeño de los productos en términos de ingresos, con el objetivo de identificar cuáles son los principales generadores de ventas y evaluar si el revenue está impulsado por volumen o por precio.

In [5]:
query = """
SELECT 
    p.ProductName,
    ROUND(SUM(od.Quantity * p.Price), 2) AS Revenue
FROM OrderDetails od
JOIN Products p ON od.ProductID = p.ProductID
GROUP BY p.ProductName
ORDER BY Revenue DESC
LIMIT 5;
"""
run_query(query)

,ProductName,Revenue
0,Côte de Blaye,62976.50
1,Thüringer Rostbratwurst,20796.72
2,Raclette Courdavault,19030.00
3,Tarte au sucre,16022.50
4,Camembert Pierrot,14620.00


**Análisis:**

El producto *Côte de Blaye* se posiciona como el principal generador de ingresos, representando aproximadamente el 16% del revenue total del negocio.

Se observa una diferencia significativa entre el primer producto y el resto del top, lo que sugiere una dependencia parcial de productos de alto desempeño.

In [6]:
query = """
SELECT 
    p.ProductName,
    p.Price,
    SUM(od.Quantity) AS UnidadesVendidas,
    ROUND(SUM(od.Quantity * p.Price), 2) AS Revenue
FROM OrderDetails od
JOIN Products p ON od.ProductID = p.ProductID
GROUP BY p.ProductName
ORDER BY Revenue DESC
LIMIT 5;
"""
run_query(query)

,ProductName,Price,UnidadesVendidas,Revenue
0,Côte de Blaye,263.50,239,62976.50
1,Thüringer Rostbratwurst,123.79,168,20796.72
2,Raclette Courdavault,55.00,346,19030.00
3,Tarte au sucre,49.30,325,16022.50
4,Camembert Pierrot,34.00,430,14620.00


**Análisis:**

El producto con mayor revenue (*Côte de Blaye*) no es el que presenta mayor volumen de ventas, sino el que posee el precio unitario más alto.

Esto indica que el revenue del negocio está impulsado en gran medida por productos premium, mientras que otros productos con mayor volumen contribuyen con menores ingresos debido a su menor precio unitario.

## 4. Análisis por categorías

En esta sección se analizan las ventas a nivel de categoría de producto, con el fin de identificar cuáles concentran la mayor parte del revenue y evaluar el grado de dependencia del negocio en determinadas líneas de producto.

In [7]:
query = """
SELECT 
    c.CategoryName,
    ROUND(SUM(od.Quantity * p.Price), 2) AS Revenue
FROM OrderDetails od
JOIN Products p ON od.ProductID = p.ProductID
JOIN Categories c ON p.CategoryID = c.CategoryID
GROUP BY c.CategoryName
ORDER BY Revenue DESC;
"""
run_query(query)

,CategoryName,Revenue
0,Beverages,99464.50
1,Dairy Products,69921.00
2,Confections,54909.16
3,Meat/Poultry,51676.52
4,Condiments,35071.60
5,Seafood,29652.30
6,Produce,23401.40
7,Grains/Cereals,22327.75


**Análisis:**

La categoría *Beverages* es la principal generadora de ingresos del negocio, concentrando aproximadamente el 26% del revenue total.

Junto con *Dairy Products*, ambas categorías representan cerca del 44% de las ventas, lo que indica una dependencia moderada del portafolio principal, aunque sin una concentración extrema en una sola categoría.

## 5. Análisis geográfico

En esta sección se analiza la distribución de las ventas por país, con el objetivo de identificar los principales mercados del negocio y evaluar el nivel de diversificación geográfica de los ingresos.

In [8]:
query = """
SELECT 
    cu.Country,
    ROUND(SUM(od.Quantity * p.Price), 2) AS Revenue
FROM OrderDetails od
JOIN Orders o ON od.OrderID = o.OrderID
JOIN Customers cu ON o.CustomerID = cu.CustomerID
JOIN Products p ON od.ProductID = p.ProductID
GROUP BY cu.Country
ORDER BY Revenue DESC;
"""
run_query(query)

,Country,Revenue
0,USA,69611.75
1,Austria,51671.96
2,Germany,47241.82
3,Brazil,40215.25
4,Canada,31326.35
5,France,29549.15
6,Denmark,17870.85
7,UK,16695.79
8,Ireland,15391.02
9,Venezuela,13556.28


**Análisis:**

Estados Unidos se posiciona como el principal mercado del negocio, seguido por Austria y Alemania.

Los cinco países con mayor revenue concentran una proporción significativa de las ventas totales, sin embargo, el negocio mantiene presencia comercial en más de 20 países, lo que evidencia una diversificación geográfica amplia y reduce la dependencia de un único mercado.

## 6. Desempeño por empleados

En esta sección se analiza el desempeño comercial de los empleados en términos de ingresos generados, con el objetivo de identificar a los principales contribuyentes al revenue del negocio.

In [9]:
query = """
SELECT 
    e.FirstName || ' ' || e.LastName AS Employee,
    ROUND(SUM(od.Quantity * p.Price), 2) AS Revenue
FROM OrderDetails od
JOIN Orders o ON od.OrderID = o.OrderID
JOIN Employees e ON o.EmployeeID = e.EmployeeID
JOIN Products p ON od.ProductID = p.ProductID
GROUP BY Employee
ORDER BY Revenue DESC;
"""
run_query(query)

,Employee,Revenue
0,Margaret Peacock,105696.50
1,Nancy Davolio,57690.39
2,Janet Leverling,42838.35
3,Robert King,39772.30
4,Laura Callahan,39309.38
5,Andrew Fuller,32503.16
6,Steven Buchanan,27480.80
7,Michael Suyama,25399.25
8,Anne Dodsworth,15734.10


**Análisis:**

Margaret Peacock se destaca como la empleada con mayor generación de ingresos, representando una proporción significativa del revenue total.

Se observa una diferencia notable entre los primeros empleados y el resto del equipo, lo que sugiere que el desempeño comercial no está distribuido de manera homogénea y podría existir oportunidad para estandarizar mejores prácticas entre el equipo de ventas.

## 7. Evolución temporal de las ventas

En esta sección se analiza la evolución de las ventas a lo largo del tiempo, con el objetivo de identificar tendencias, patrones estacionales y cambios en el desempeño del negocio.

In [10]:
query = """
SELECT 
    strftime('%Y-%m', o.OrderDate) AS YearMonth,
    ROUND(SUM(od.Quantity * p.Price), 2) AS Revenue
FROM OrderDetails od
JOIN Orders o ON od.OrderID = o.OrderID
JOIN Products p ON od.ProductID = p.ProductID
GROUP BY YearMonth
ORDER BY YearMonth;
"""
run_query(query)

,YearMonth,Revenue
0,1996-07,37779.85
1,1996-08,33285.49
2,1996-09,34565.60
3,1996-10,51528.69
4,1996-11,62163.99
5,1996-12,63721.23
6,1997-01,83400.47
7,1997-02,19978.91


**Análisis:**

Se observa una tendencia creciente en las ventas desde julio de 1996 hasta enero de 1997, alcanzando su punto máximo en enero de 1997.

Los meses de noviembre y diciembre presentan un incremento relevante en el revenue, lo que sugiere un posible efecto estacional asociado al cierre de año. Posteriormente, en febrero de 1997 se evidencia una caída significativa en las ventas, lo que podría estar relacionado con un período de menor demanda.

## 8. Conclusiones finales

- El negocio presenta un revenue total cercano a **386 mil unidades monetarias**, con un ticket promedio por orden de aproximadamente **1.970**, lo que indica un valor de compra relativamente alto.

- El revenue está fuertemente impulsado por productos de precio elevado, destacándose *Côte de Blaye* como el principal generador de ingresos, lo que evidencia una estrategia basada en productos premium.

- A nivel de categorías, *Beverages* y *Dairy Products* concentran una parte significativa de las ventas, aunque el portafolio mantiene una distribución equilibrada entre varias líneas de producto.

- Desde el punto de vista geográfico, el negocio cuenta con una amplia diversificación internacional, con Estados Unidos, Austria y Alemania como los mercados más relevantes.

- El desempeño por empleados muestra diferencias marcadas en la generación de ingresos, lo que sugiere oportunidades para estandarizar prácticas comerciales exitosas dentro del equipo.

- El análisis temporal evidencia una tendencia de crecimiento hacia finales de 1996, con un pico en enero de 1997 y un descenso posterior, lo que podría estar asociado a patrones estacionales.